In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt 
import os
import functools
import numpy as np
import random as rndm
from tqdm import tqdm
import pandas as pd
from PIL import Image
import cv2
import h5py
import sys
import threading
import glob
from IPython import display as ipythondisplay
from string import Formatter
import time
from datetime import date
from mtcnn import MTCNN
import socket
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
def test_images():  
  model = tf.keras.models.load_model('face.h5') 

  # Create a test data generator
  test_datagen = ImageDataGenerator(rescale=1./255) 

  # Path to the test directory
  test_dir = 'C:/Users/Gishan/Documents/HS/faces' 

  # Image dimensions
  img_height, img_width = 150, 150 
  batch_size = 32

# Create test data generator
  test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary', 
    shuffle=False 
  )

# Get predictions
  predictions = model.predict(test_generator)

# Convert probabilities to class labels (0 or 1)
  predicted_classes = (predictions > 0.5).astype(int) 

# Get true class labels
  true_labels = test_generator.classes

# Evaluate model performance
  accuracy = accuracy_score(true_labels, predicted_classes)
  conf_matrix = confusion_matrix(true_labels, predicted_classes)
  classification_rep = classification_report(true_labels, predicted_classes)

# Print results 
  print(f"Test Accuracy: {accuracy:.2f}") 
  print("Confusion Matrix:\n", conf_matrix)
  print("Classification Report:\n", classification_rep)

  delete_files_in_directory('C:/Users/Gishan/Documents/HS/faces/user_1')

  return accuracy

In [ ]:
def delete_files_in_directory(directory_path):
    if not os.path.exists(directory_path): # Check if the directory exists
        raise FileNotFoundError(f"Directory '{directory_path}' not found.") #Error if not

    for filename in os.listdir(directory_path): # Loop through all files in the directory
        file_path = os.path.join(directory_path, filename) # Get the full path of the file
        os.remove(file_path) #remove the file

def mse(imageA, imageB): # Function to calculate the mean squared error between two images
    err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2) 
    err /= float(imageA.shape[0] * imageA.shape[1])
    return err


In [ ]:
ESP_IP = '192.168.8.188'  #IP of the ESP32 board
ESP_PORT = 80 #port of the ESP board

def send_data_to_esp(data):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s: 
      s.connect((ESP_IP, ESP_PORT)) #Connect to the ESP32 board
      s.sendall(data.encode()) #Send data to the ESP

In [ ]:
def userinput():
    x=input('Enter any key to stop \n') #user input
    return x

thread = threading.Thread(target=userinput) #Start thread to parallely to controll the process

thread.start()

cap = cv2.VideoCapture('http://192.168.8.168:81/stream') #capturing the stream

detec = MTCNN()

# Check if camera opened successfully
if not cap.isOpened():
    print("Cannot open camera")
    exit()

current=date.today() #Grab the date

data_directory = 'C:/Users/Gishan/Documents/HS/' #Directory to save the data

fourcc = cv2.VideoWriter_fourcc(*'mp4v')  #Video record format
print('recording..')

n=0
file_name = f'{current} {n}.mp4' 
file_path = os.path.join(data_directory, file_name)

while os.path.exists(file_path)==True:
    n+=1
    file_name = f'{current} {n}.mp4' 
    file_path = os.path.join(data_directory, file_name)

ind=0
filename = f'img{ind}.jpg' 
data_directory = './frames'
file_path = os.path.join(data_directory, filename)

while os.path.exists(file_path)==True:
    ind+=1
    filename = f'img{ind}.jpg' 
    file_path = os.path.join(data_directory, filename)

ind-=1

out = cv2.VideoWriter(file_name, fourcc, 10.0, (640, 480)) #Video writer to save the video
data = '1'

num=0
while thread.is_alive(): #if thread is alive 
    ret, frame = cap.read()
    i=1

    while not ret: #if there is no frame return
        ret, frame = cap.read()
        if i==1:
            print('wait')
            i+=1
    
    out.write(frame) #Write the frame to the video

    if current != date.today(): #current date is different from the initial date
        current = date.today() 
        out = cv2.VideoWriter(file_name, fourcc, 10.0, (640, 480)) #reintialize the details of the recording

    if cv2.waitKey(1) == ord('q'):
        break
    
    faces = detec.detect_faces(frame) #detect faces in the frame

    for face in faces:
        x, y, w, h=face['box'] #get the position of the face
        face_roi = frame[y:y+h, x:x+w] #draw the square
        cv2.imwrite(f'C:/Users/Gishan/Documents/HS/faces/user_1/img{num}.jpg',face_roi) #store the face in local storage
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2) 
    
    cv2.imshow('Camera', frame)    

    num+=1    
    if num==10: #count the frame till 10
        num=0 #Reset
        data = '1' #Reset the data
        if len(os.listdir('./faces/user_1'))>0: #if there are faces in the local storage
            accuracy=test_images()  #use the CNN model to test the faces
            print(f'accuracy: {accuracy}')
            if accuracy>=0.75: #if the accuracy is greater than 75%
                data = '0' 
            else:
                cv2.imwrite(f'C:/Users/Gishan/Documents/HS/frames/img{ind}.jpg',frame) #record the face in the local storage
        send_data_to_esp(data) #send data to the ESP32

out.release()
cap.release()
cv2.destroyAllWindows()
thread.join()

In [ ]:
#Reset ESP
send_data_to_esp('1')
delete_files_in_directory('C:/Users/Gishan/Documents/HS/faces/user_1')

In [ ]:
#misc code used to create validate directory
detec = MTCNN()
for i in range(100000, 100300):
    prev=cv2.imread(f'./dataset/img_align_celeba/img_align_celeba/{i}.jpg')
    faces = detec.detect_faces(prev)
    
    
    for face in faces:
        x, y, w, h=face['box']
        face_roi = prev[y:y+h, x:x+w] 
        cv2.imwrite(f'./faces/intruder/{i}.jpg', face_roi)
        

cv2.waitKey(0)    
cv2.destroyAllWindows()

In [ ]:
#sequential model to learn face features
n_filters=16
def learn_face(n_outputs=1):
    #2D Layer which make 2D convolution
    Conv2D = functools.partial(tf.keras.layers.Conv2D, padding='same', activation='relu') 
    #batch normalization rescalling
    BatchNormalization = tf.keras.layers.BatchNormalization
    #Flatten the layer multi-dimension to single
    Flatten = tf.keras.layers.Flatten
    #dense layer fully connected layer
    Dense = functools.partial(tf.keras.layers.Dense, activation='relu')

    model = tf.keras.Sequential([
        Conv2D(filters=1*n_filters, kernel_size=5,  strides=2),
        BatchNormalization(),
        Conv2D(filters=2*n_filters, kernel_size=5,  strides=2),
        BatchNormalization(),
        Conv2D(filters=4*n_filters, kernel_size=5,  strides=2),
        BatchNormalization(),
        Conv2D(filters=8*n_filters, kernel_size=5,  strides=2),
        BatchNormalization(),
        Conv2D(filters=16*n_filters, kernel_size=5,  strides=2),
        BatchNormalization(),
        
        Flatten(),

        Dense(512),
        Dense(n_outputs, activation=None),
    ])
    return model
face = learn_face()

# Compile the model
face.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [ ]:
detect=MTCNN()
#enroll new faces
def enroll_faces():
    n=1
    x=True
    cap = cv2.VideoCapture('http://192.168.8.168:81/stream') # capture video from the camera
    precentage=0
    while x:
        ret, frame = cap.read()
        faces = detect.detect_faces(frame)

        for face in faces:
            x, y, w, h=face['box']
            face_roi = frame[y:y+h, x:x+w] 
            cv2.imwrite(f'./faces/user_1/img{precentage}.jpg', face_roi)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            precentage+=1
        
        cv2.imshow('frame', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        if precentage<=100:
            print('=', end='')
        else:
            break
              
    cv2.destroyAllWindows()
    cap.release()

    train()

In [ ]:
enroll_faces()

In [ ]:
def train():
    img_height, img_width = 150, 150 
    batch_size = 10

    train_dir = './faces' 
    val_dir = './nonuser' 


    train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
    )

    val_datagen = ImageDataGenerator(rescale=1./255)

# Load and preprocess images from directories
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='binary'
    )

    validation_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='binary'
    )

# Train the model
    history = face.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // batch_size,
        epochs=10, 
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // batch_size
    )

# Evaluate the model
    test_loss, test_acc = face.evaluate(validation_generator)
    print('Test accuracy:', test_acc)

    # Save the model
    face.save('face.h5') 

In [ ]:
train()

In [ ]:
os.remove('face.h5') #delete model

In [ ]:
# Define a convolutional layer with glorot_uniform kernel
conv_layer = tf.keras.layers.Conv2D(
    filters=1,  # Single filter
    kernel_size=(3, 3),  # 3x3 kernel
    strides=(1, 1),  # Stride of 1
    padding='same',  # Preserve spatial dimensions
    kernel_initializer='glorot_uniform',  # Use glorot_uniform initializer
    use_bias=False  # Disable bias for simplicity
)

In [ ]:
# Create a synthetic grayscale image (e.g., a checkerboard pattern)
image = cv2.imread(f'C:/Users/Gishan/Documents/HS/frames/out2.jpg')

# Display the original image
plt.imshow(image, cmap='gray')
plt.title("Original Image")
plt.colorbar()
plt.show()

image=image.astype(np.float32)
# Build the layer by specifying the input shape
input_image = tf.expand_dims(image,axis=0)# Add batch and channel dimensions
conv_layer.build(input_image.shape)

# Apply the convolutional layer to the image
output_image = conv_layer(input_image)
#output_image = conv_layer(output_image)
#output_image = conv_layer(output_image)

# Remove batch and channel dimensions for visualization
output_image = tf.squeeze(output_image).numpy()
cv2.imwrite(f'C:/Users/Gishan/Documents/HS/frames/out3.jpg',output_image)

# Display the output image
plt.imshow(output_image, cmap='gray')
plt.title("Output Image (After Applying glorot_uniform Kernel)")
plt.colorbar()
plt.show()

# Get the kernel weights
kernel_weights = conv_layer.weights[0].numpy()

# Visualize the kernel
plt.imshow(output_image, cmap='gray')  # Visualize the first (and only) filter
plt.title("glorot_uniform Kernel (3x3)")
plt.colorbar()
plt.show()